Defining data frame variables 

In [1]:
import pandas as pd
import import_ipynb
import removeStopWords
import io

data = removeStopWords.removeStopWords(open("../data/train_self_original.txt", "r").read())
buf = io.StringIO(msg)
rowList = buf.readlines()

conversations = {
    "conv_id": []
}
chats = {
    "conv_id": [],
    "buddy1": [],
    "chat_id": [],
    "buddy2": []
}
personals = {
    "persona": [],
    "personaId": [],
    "conv_id": []
}
answers = {
    "conv_id": [],
    "chat_id": [],
    "ans_id": [],
    "answer": []
}
isPersona = lambda rowContext : rowContext.find("your persona:")!=-1


importing Jupyter notebook from removeStopWords.ipynb


ValueError: [E088] Text of length 112522819 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [ ]:
def parseAnswer(conv_id, chat_id, answersContext):
    returnind = answersContext.index("\n")
    answersContext=answersContext[0:returnind]
    answersList = answersContext.split("|")
    for i in range(0, len(answersList)):
        answers["conv_id"].append(conv_id)
        answers["chat_id"].append(chat_id)
        answers["ans_id"].append(len(answers["ans_id"])+1)
        answers["answer"].append(answersList[i])        


In [ ]:
def parseChat(currentContext, conv_id):
    buddy1EndPointPattern = "\t"
    buddy1EndPoint = currentContext.index(buddy1EndPointPattern)
    buddy2EndPointPattern ="\t\t"
    buddy2EndPoint = currentContext.index(buddy2EndPointPattern)
    buddy1str = currentContext[2:buddy1EndPoint]
    chats["conv_id"].append(conv_id)
    chats["buddy1"].append(buddy1str)
    chats["chat_id"].append(len(chats["chat_id"])+1)
    chats["buddy2"].append(currentContext[buddy1EndPoint+len(buddy1EndPointPattern):buddy2EndPoint])
    answersContext = currentContext[buddy2EndPoint+len(buddy2EndPointPattern):]
    parseAnswer(conv_id, chats["chat_id"][-1], answersContext)
    


In [ ]:
def parsePersona(conv_id, persona):
    personals["persona"].append(slicePersona(persona))
    personals["personaId"].append(len(personals["personaId"])+1)
    personals["conv_id"].append(conv_id)

def slicePersona(persona) :
    pattern = "your persona:"
    prefixInd = persona.rindex(pattern)
    retInd = persona.index("\n")
    return persona[prefixInd+len(pattern):retInd]

In [ ]:
def parseConversation(i):
    personaLen=0
    personaIt = i
    while(isPersona(rowList[personaIt]) and personaIt < len(rowList)):
        personaLen+=1
        personaIt+=1
    conversations["conv_id"].append(len(conversations["conv_id"])+1)
    
    for persona in rowList[i:i+personaLen] :
        parsePersona(conversations["conv_id"][-1], persona)
    return personaLen-1
    

Start converting information from txt file and creating dataframes

In [ ]:
def parseObjects():
    skipPersona = lambda index, personaLen : index+int(personaLen)
    i = 0
    
    while i < len(rowList):
        if isPersona(rowList[i]) :
            personaLen = parseConversation(i)
            i = skipPersona(i, personaLen)
        else:
            parseChat(rowList[i], conversations["conv_id"][-1])
        i+=1

In [ ]:
parseObjects()
convdf = pd.DataFrame(conversations, columns=["conv_id"])
personadf = pd.DataFrame(personals, columns=["conv_id", "persona","personaId"])
chatdf = pd.DataFrame(chats, columns=["conv_id", "buddy1", "chat_id", "buddy2"])
answerdf = pd.DataFrame(answers, columns = ["conv_id", "chat_id", "ans_id", "answer"])
    


Getting the most common personal informations

In [ ]:
answerdf